In [14]:
import pandas as pd

# 엑셀 파일 경로
fs_path = r'C:\Users\rkddk\OneDrive\문서\카카오톡 받은 파일\perdata.xlsx'
invest_df = pd.read_excel(fs_path)

# 종목코드가 6자리가 되도록 앞에 0을 채우는 함수
def pad_stock_code(df, column_name):
    df[column_name] = df[column_name].astype(str).str.zfill(6)
    return df
c
# 데이터프레임에 종목코드 6자리 맞추기 적용
invest_df = pad_stock_code(invest_df, '종목코드')

def get_value_rank(invest_df, value_type, index_date, num):
    # 기준일을 필터링하여 특정 날짜의 데이터만 선택
    df_filtered = invest_df[invest_df['기준일'] == index_date]
    
    # value_type 컬럼을 숫자로 변환
    df_filtered[value_type] = pd.to_numeric(df_filtered[value_type], errors='coerce')
    
    # value_type 기준으로 정렬
    value_sorted = invest_df.sort_values(by=value_type)
    
    # 순위 열 추가
    value_sorted[value_type + '순위'] = value_sorted[value_type].rank()
    
    # 결과 데이터프레임에서 필요한 열만 선택하여 반환
    result = value_sorted[['종목코드', value_type, value_type + '순위']].reset_index(drop=True)
    return result[:num] if num else result

# 테스트 함수 호출
result = get_value_rank(invest_df, 'PSR','2022/12', 20)
result.set_index('종목코드', inplace=True)
# 결과 출력
print(result)


         PSR  PSR순위
종목코드               
000880  0.05    1.0
036460  0.06    2.0
001120  0.07    4.5
267250  0.07    4.5
047050  0.07    4.5
001040  0.07    4.5
139480  0.09    7.0
001740  0.10    8.0
034730  0.11    9.0
006260  0.13   10.5
006040  0.13   10.5
006360  0.15   13.0
078930  0.15   13.0
004020  0.15   13.0
011210  0.16   15.0
020560  0.17   17.0
047040  0.17   17.0
298020  0.17   17.0
375500  0.18   20.5
000120  0.18   20.5


In [25]:
def make_value_combo(value_list, invest_df, index_date, num):
    for i, value in enumerate(value_list):
        temp_df = get_value_rank(invest_df, value, index_date, None)
        if i == 0:
            value_combo_df = temp_df
            rank_combo = temp_df[value + '순위'].copy()
        else:
            temp_df.set_index('종목코드', inplace=True)
            value_combo_df = pd.merge(value_combo_df, temp_df, on='종목코드', how='outer', suffixes=('', '_' + value))
            rank_combo += value_combo_df[value + '순위']
    
    # 종합 순위 계산 및 적용
    value_combo_df['종합 순위'] = rank_combo.rank(method='min')
    value_combo_df.sort_values('종합 순위', inplace=True)
    value_combo_df.set_index('종목코드', inplace=True)
    return value_combo_df.head(num)


In [26]:
# 테스트 함수 호출
result = make_value_combo(['PER', 'PBR'], invest_df, '2022/12', 20)
# 결과 출력
print(result)

         PER  PER순위   PBR  PBR순위  종합 순위
종목코드                                   
000880  2.14    5.0  0.23    4.0    1.0
001230  2.27    7.0  0.28    8.0    2.0
139480  2.65    9.0  0.24    6.0    2.0
003240  3.24   15.0  0.19    1.0    4.0
078930  1.94    4.0  0.33   13.5    5.0
036460  2.24    6.0  0.32   11.5    5.0
000670  3.04   12.0  0.28    8.0    7.0
004020  4.01   20.0  0.22    2.0    8.0
010060  1.74    3.0  0.39   20.5    9.0
016380  1.38    2.0  0.44   25.5   10.0
375500  3.33   16.0  0.33   13.5   11.0
011200  0.95    1.0  0.46   28.5   11.0
006260  2.83   10.0  0.50   35.5   13.0
047040  3.45   18.0  0.46   28.5   14.0
006360  5.34   30.0  0.37   19.0   15.0
005440  7.37   50.0  0.23    4.0   16.0
001120  2.55    8.0  0.57   46.5   17.0
161390  5.59   34.0  0.44   25.5   18.0
267250  3.20   14.0  0.57   46.5   19.0
057050  7.49   52.0  0.32   11.5   20.0


In [27]:
make_value_combo(['PER', 'PBR', 'PSR', 'PCR'], invest_df, '2015/12', 20)

,PER,PER순위,PBR,PBR순위,PSR,PSR순위,PCR,PCR순위,종합 순위
종목코드,,,,,,,,,
000880,2.14,5.0,0.23,4.0,0.05,1.0,0.99,4.0,1.0
036460,2.24,6.0,0.32,11.5,0.06,2.0,1.03,6.0,2.0
139480,2.65,9.0,0.24,6.0,0.09,7.0,1.05,7.0,3.0
078930,1.94,4.0,0.33,13.5,0.15,13.0,1.94,17.0,4.0
004020,4.01,20.0,0.22,2.0,0.15,13.0,1.57,14.0,5.0
016380,1.38,2.0,0.44,25.5,0.19,24.0,1.28,8.0,6.0
001120,2.55,8.0,0.57,46.5,0.07,4.5,1.76,15.0,7.0
006260,2.83,10.0,0.50,35.5,0.13,10.5,1.98,18.0,7.0
000670,3.04,12.0,0.28,8.0,0.25,34.5,2.28,21.5,9.0
